# Assessment 1 Parsing Data And Text Preprocessing
## Task 1: Parsing Text Files
*   Name: Jingyi Chen
*   ID: 31314295

### 1.0 Initializing the Environment
> Load the google drive path to read the data source;

> Import the packages needed for programming. 

In [75]:
from google.colab import drive
drive.mount('/content/drive') 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [76]:
import re
import os

In [77]:
# Load the data
data = ""
directory = "/content/drive/Shareddrives/FIT5196-s1-2022/A1/Task1/input_data/31314295/"
for file in os.listdir(directory):
    f = open(directory + file, 'r')
    data += f.read()


### 2.0 Detected All The Tag Names

Call the re model of python and use the findall method to filter the data in the xml file. Use regular expressions to select all tag names, and then remove the duplicates to get all the tag name information.

> Regular Expression: **\\$(.*?)\\.:**

> Regular Expression Description: Due to we want to match the data in '<>' tags like users, reviews, summary and others. Like '$tags.:' this kind of data. So we use Regular Expression: **\\$(.*?)\\.:**. To prevent special characters from being transferred, we use backslashes to distinguish between these characters(\$ and \.). And (.*?) these characters is to match any character. 











In [78]:
# Get all the tag names by regular expression
reg_name_tags = re.compile('\$(.*?)\.:')
uni_tags = re.findall(reg_name_tags, data)
uni_tags = set(uni_tags)
uni_tags

{'1 in 1913, now costs more than $50.Prior to 1913, if the government needed additional money, it could simply print the money itself. Abraham Lincoln did this during the Civil War when "Greenbacks" were issued. These were simply promissory notes like today\'s Federal Reserve Notes that we think of as money. If you look at the top of any bill, you will see the words Federal Reserve Note. However, after the passage of the Federal Reserve Act, the government must now have a third party, the Fed, print the additional money it needs. This might not be such a horrible thing except for the fact that the government must now pay interest on this newly created money that it has borrowed from the Fed. In addition, this new money is the product of purchasing power that has been stolen from the people of the United States. So, in effect, the taxpayers must now pay interest on money that the Fed has stolen from them, and then loaned back to them.But wait. It gets worse.After the government borrows 

As the results show, there are many different tag names, except for the first sentence, which is an accidental match, three different cases for product ID, three different cases for review ID, and so on. When we need to rematch, we take all of these things into account so that we don't lose data.

---



### 3.0 Defining Funtions and Regular Expressions

#### 3.1 Defining Regular Expressions

Define different regular expressions for different tags in order to extract data from those tags. Use the re.compile method to compile regular expressions for the following seven different tags, and then use it when extracting data.

Regular expression syntax description:
>  \w+ Matches any alphanumeric character with many times

>  \d+ Matches any decimal digit with many times

>  A | B means to match to the two cases can be A or B 

> Using backslashes to distinguish between these characters(\$ and \.)


In [79]:
# define the regular expressions for different tags
reg_rev_id = re.compile('\$(rev_ID|reviewer.ID|reviewerID)\.: (\w+)')
reg_prod_id = re.compile('\$(prod_ID|product.ID|productID)\.: (\w+)')
reg_rev_date = re.compile('\$(reviewDate|review_date)\.: (\d+ \d+, \d+)')
reg_rev_help = re.compile('\$(Helpful\?|No\. helps)\.: (\[\d+, \d+\])')
reg_rev_text = re.compile('\$(REVIEW|review_text)\.: ')
reg_rev_summ = re.compile('\$(review_summary|SUMMARY)\.: ')
reg_rev_name = re.compile('\$(rev_NAME|reviewer.NAME|reviewerName)\.: ')

In [80]:
# check the number of rows in different regular expressions
for each in [reg_rev_name, reg_rev_id, reg_prod_id, reg_rev_date, reg_rev_help, reg_rev_text, reg_rev_summ]:
    print(each, len(re.findall(each, data)))

re.compile('\\$(rev_NAME|reviewer.NAME|reviewerName)\\.: ') 40000
re.compile('\\$(rev_ID|reviewer.ID|reviewerID)\\.: (\\w+)') 40000
re.compile('\\$(prod_ID|product.ID|productID)\\.: (\\w+)') 40000
re.compile('\\$(reviewDate|review_date)\\.: (\\d+ \\d+, \\d+)') 40000
re.compile('\\$(Helpful\\?|No\\. helps)\\.: (\\[\\d+, \\d+\\])') 40000
re.compile('\\$(REVIEW|review_text)\\.: ') 40000
re.compile('\\$(review_summary|SUMMARY)\\.: ') 40000


These results show the number of rows of different regular expressions. All these regular expressions have 40000 rows. 


---



#### 3.2 Defining Funtions

Get data for different regular expressions: using the regular expression to split the data, and get the data from the tags

> Date list type ('review_date', '03 28, 2013')

> prod_id list type ('product.ID', 'B005DOK8NW')

> helps list type ('Helpful? ', '[0, 0]')

In [81]:
def extract_data(data):
    rev_dates = re.findall(reg_rev_date, data)[0][1]
    prod_ids = re.findall(reg_prod_id, data)[0][1]
    rev_helps = re.findall(reg_rev_help, data)[0][1]
    
    # Extract and split the data from tags
    data_copy = data
    data_copy = re.split(reg_rev_date, data_copy)
    data_copy = [code for each in data_copy for code in re.split(reg_prod_id, each)]
    data_copy = [code for each in data_copy for code in re.split(reg_rev_help, each)]
    data_copy = [code for each in data_copy for code in re.split(reg_rev_id, each)]
    data_copy = [code for each in data_copy for code in re.split(reg_rev_text, each)]
    data_copy = [code for each in data_copy for code in re.split(reg_rev_summ, each)]
    data_copy = [code for each in data_copy for code in re.split(reg_rev_name, each)]
    
    # Double-layer for loop retrieves review and summary data into an array
    rev_txt_list = [data_copy[i+1] for i in range(len(data_copy))  if data_copy[i] in ['REVIEW', 'review_text']]
    rev_summ_list = [data_copy[i+1] for i in range(len(data_copy))  if data_copy[i] in ['review_summary', 'SUMMARY']]
        
    return rev_dates, prod_ids, rev_helps, rev_txt_list[0], rev_summ_list[0]

_format_date function description:


> Adding 0 in month and day for using in next part to compare the dates.



In [82]:
def _format_date(date):
    date_list = date.split()
    # YEAR MONTH DAY
    date_list = [date_list[-1], date_list[0], date_list[1]]

    # Make sure all dates are in one format: 20220101
    if len(date_list[-1]) < 3:
      date_list = [date_list[0], date_list[1], '0' + date_list[-1]]
    if len(date_list[1]) < 2:
      date_list = [date_list[0], '0' + date_list[1], date_list[-1]]
    return date_list

correct_content function description:

> This method is used to replace special characters, null values, symbols, and NA values in data



In [83]:
## remove all the illegel characters
def correct_content(context):
    context = re.sub('&', '&amp;', context)
    context = re.sub('', '', context)
    re.sub('[\\x00-\\x08\\x0b-\\x0c\\x0e-\\x1f]','',context)
    context = re.sub('&#27;', '', context)
    context = re.split('\n\n\n', context)
    if context[-1] == "" or context[-1] == '\n':
        context = context[:-1]
    context = '. '.join(context)
    context = re.split('\n', context)
    context = [each for each in context if each != ""]
    context = '. '.join(context)
    context = re.sub("\'", '&#39;', context)
    context = re.sub("^K", ' ', context)
    if context == 'nan':
        context = 'NAN'    
    return context

### 4.0 Extract The Data From Tags



#### 4.1 Extract reviewer_id data From Tags

Use the review_id regular expression to **split** the match by calling the re.split method. Then data cleaning involves **removing null values and newlines**. Finally, through the for loop, all the values of a **Reviewer ID** were placed in a list, and all the values without a Reviewer ID were placed in a list. 

These two lists are used in next part. rev_ids list is used as the key to match the answer format. and the context list is used to extract the other data from tags.

In [84]:
data_copy = data
data_copy = re.split(reg_rev_id, data_copy)
data_copy = [each for each in data_copy if each not in ['\n', "", ' ']] # remove the empty value and \n value

contexts = [data_copy[i] for i in range(2, len(data_copy), 3)] # except review_id
all_rev_ids = [data_copy[i] for i in range(1, len(data_copy), 3)] # review id
print(all_rev_ids[0:10])

['A30YO2G0MDE1PG', 'A1TI7MZLZSEO4I', 'A2WVFAQUUWB94K', 'A5NNSSZRMVFMJ', 'A3KDP3BJ7GF5NT', 'A39DQIQ1GO2J4Y', 'AYNAH993VDECT', 'A12PZF121RG0HH', 'A3J6NW3S55DWLL', 'A3GCX2GDJ67ZWA']


#### 4.2 Extract All Data from Tags and Transfer to Json Format

Through the **for loop**, loop through the Reviewer ID and other fields. With reviewer ID as the key, other fields were nested in JSON and stored in the dictionary. If the Reviewer ID already exists and needs to compare dates, use the date conversion function **_format_date** to convert the dates, and then take the larger date as **latest_review_date**. Then add the JSON data for the Reviews

In [85]:
dic = {}
for i in range(len(all_rev_ids)):
    rev_id = all_rev_ids[i]
    cont = contexts[i]
    date, prod_id, helps, review_text, summary = extract_data(cont)
    if rev_id not in dic:
        dic[rev_id] = \
        {
            'latest_review_date':  date,
            'reviews':[{
                    'productID': prod_id,
                    'review_date': date, 
                    'review_helpful': helps, 
                    'review_text':  correct_content(review_text),
                    'review_summary':  correct_content(summary)
                    }]
        }
    else:
        new_dic = {
                    'productID': prod_id,
                    'review_date': date, 
                    'review_helpful': helps, 
                    'review_text':  correct_content(review_text),
                    'review_summary':  correct_content(summary)
                    }
        old_date = _format_date(dic[rev_id]['latest_review_date'])
        new_date = _format_date(date)
        if new_date < old_date:
            date = dic[rev_id]['latest_review_date']
        
        dic[rev_id] = \
        {
            'latest_review_date': date,
            'reviews': dic[rev_id]['reviews'] + [new_dic]
        }

### 5.0 Writing the Final Results to the File
The code here is mainly used to convert the dictionary data into a file. Due to the specific format of assessment, we need to **concatenate some strings** such as **'<users>'**. Concatenate these strings with the data in the dictionary to get the final data, iterate through all the data through a **double-layer for loop**, and finally write it out to the file.

In [86]:
with open('/content/drive/MyDrive/2020 August - Master of Data Science/31314295.xml', 'w') as d:
    d.write('<users>' + '\n')
    for _id in dic:
        date = dic[_id]['latest_review_date']
        reviews = dic[_id]['reviews']
        
        d.write('<user name="' + _id + '">' + '\n')
        d.write('<latest_review_date>' + date + '</latest_review_date>' + '\n')
        d.write('<reviews>' + '\n')
        for each in reviews:
            prod_id = each['productID']
            review_date = each['review_date']
            review_helpful = each['review_helpful']
            review_text = each['review_text']
            review_summary = each['review_summary']
            d.write('<review>' + '\n')
            
            d.write('<productID>' + prod_id)
            d.write('</productID>' + '\n')
            d.write('<review_date>' + review_date)
            d.write('</review_date>' + '\n')
            d.write('<review_helpful>' + review_helpful)
            d.write('</review_helpful>' + '\n')
            d.write('<review_text>' + review_text)
            d.write('</review_text>' + '\n')
            d.write('<review_summary>' + review_summary)
            d.write('</review_summary>' + '\n')
            d.write('</review>' + '\n')
        d.write('</reviews>' + '\n')
        d.write('</user>' + '\n')
    d.write('</users>')